In [3]:
import numpy as np
from tdc.multi_pred import DTI
import sys
import os
import pandas as pd

major = sys.version_info.major
minor = sys.version_info.minor
micro = sys.version_info.micro

print(f"Python version: {major}.{minor}.{micro}")

data = DTI(name='KIBA')
KibaDataSet = data.get_data()
print('number of drugs =',len(KibaDataSet['Drug_ID'].unique()))
print('number of proteins = ',len(KibaDataSet["Target_ID"].unique()))
KibaDataSet


# Path to the folder where your protein graphs are saved
graph_folder = "./ProteinGraphs"

# Get the list of protein IDs for which you have graphs (strip the '_graph.pt' suffix)
protein_ids_with_graphs = [f.split('_graph.pt')[0] for f in os.listdir(graph_folder) if f.endswith('_graph.pt')]

# Convert this list to a set for faster lookups
protein_ids_with_graphs = set(protein_ids_with_graphs)


# Assuming KibaDataSet is already loaded as a pandas DataFrame
valid_protein_dataset = KibaDataSet[KibaDataSet["Target_ID"].isin(protein_ids_with_graphs)]

# Save the new dataset with valid proteins
valid_protein_dataset.to_csv("filtered_KibaDataSet.csv", index=False)

# Check the size of the new dataset
print(f"Original dataset had {len(KibaDataSet['Target_ID'].unique())} unique proteins.")
print(f"Filtered dataset has {len(valid_protein_dataset['Target_ID'].unique())} unique proteins.")


len(valid_protein_dataset)


import os
import torch
import pickle
import pandas as pd

def load_graph(path, is_pickle=True):
    """
    Load a molecule graph (.pkl) or a protein graph (.pt).
    If is_pickle is True, use pickle to load the file; otherwise, use torch.load.
    """
    if is_pickle:
        with open(path, 'rb') as f:
            return pickle.load(f)
    else:
        return torch.load(path)

def prepare_dataset_individual_save_as_pt(filtered_dataset, molecule_graph_dir, protein_graph_dir, output_dir):
    """
    Incrementally prepares the dataset and saves each (molecule, protein, target) tuple as a separate .pt file.
    
    Args:
    - filtered_dataset: The filtered KIBA dataset (DataFrame).
    - molecule_graph_dir: Directory where molecule graphs are stored.
    - protein_graph_dir: Directory where protein graphs are stored.
    - output_dir: Directory to save the prepared dataset incrementally.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for index, row in filtered_dataset.iterrows():
        protein_id = row['Target_ID']
        chembl_id = row['Drug_ID']
        
        # Load the protein graph (.pt)
        pro_graph_path = os.path.join(protein_graph_dir, f"{protein_id}_graph.pt")
        if not os.path.exists(pro_graph_path):
            print(f"Protein graph not found: {protein_id}")
            continue
        pro_graph = load_graph(pro_graph_path, is_pickle=False)
        
        # Load the molecule graph (.pkl)
        mol_graph_path = os.path.join(molecule_graph_dir, f"{chembl_id}_graph.pkl")
        if not os.path.exists(mol_graph_path):
            print(f"Molecule graph not found: {chembl_id}")
            continue
        mol_graph = load_graph(mol_graph_path)

        # Load target (affinity value)
        target = torch.tensor([row['Y']], dtype=torch.float)
        
        # Create the sample as a tuple (molecule graph, protein graph, target)
        sample = (mol_graph, pro_graph, target)
        
        # Save the sample as a .pt file
        sample_path = os.path.join(output_dir, f"sample_{index}.pt")
        torch.save(sample, sample_path)

        if(index%10000 == 0 ):
            print(f"Saved sample {index} as {sample_path}")
            

        

# Example usage for individual saving
molecule_graph_dir = 'molecule_graphs/'  # Directory where molecule graphs are stored
protein_graph_dir = 'ProteinGraphs/'  # Directory where protein graphs are stored
filtered_dataset_path = 'filtered_KibaDataSet.csv'  # Path to the filtered dataset CSV
output_dir = 'prepared_samples/'  # Directory to save individual samples

# Load filtered dataset CSV
filtered_dataset = pd.read_csv(filtered_dataset_path)

# Prepare the dataset incrementally, saving each sample as a .pt file
prepare_dataset_individual_save_as_pt(filtered_dataset, molecule_graph_dir, protein_graph_dir, output_dir)

print("Dataset preparation completed.")


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, global_max_pool as gmp, global_add_pool as gap,global_mean_pool as gep,global_sort_pool
from torch_geometric.utils import dropout_adj


# GCN based model
class GNNNet(torch.nn.Module):
    def __init__(self, n_output=1, num_features_pro=54, num_features_mol=78, output_dim=128, dropout=0.2):
        super(GNNNet, self).__init__()

        print('GNNNet Loaded')
        self.n_output = n_output
        self.mol_conv1 = GCNConv(num_features_mol, num_features_mol)
        self.mol_conv2 = GCNConv(num_features_mol, num_features_mol * 2)
        self.mol_conv3 = GCNConv(num_features_mol * 2, num_features_mol * 4)
        self.mol_fc_g1 = torch.nn.Linear(num_features_mol * 4, 1024)
        self.mol_fc_g2 = torch.nn.Linear(1024, output_dim)

        # self.pro_conv1 = GCNConv(embed_dim, embed_dim)
        self.pro_conv1 = GCNConv(num_features_pro, num_features_pro)
        self.pro_conv2 = GCNConv(num_features_pro, num_features_pro * 2)
        self.pro_conv3 = GCNConv(num_features_pro * 2, num_features_pro * 4)
        # self.pro_conv4 = GCNConv(embed_dim * 4, embed_dim * 8)
        self.pro_fc_g1 = torch.nn.Linear(num_features_pro * 4, 1024)
        self.pro_fc_g2 = torch.nn.Linear(1024, output_dim)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

        # combined layers
        self.fc1 = nn.Linear(2 * output_dim, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.out = nn.Linear(512, self.n_output)

    def forward(self, data_mol, data_pro):
        # get graph input
        mol_x, mol_edge_index, mol_batch = data_mol.x, data_mol.edge_index, data_mol.batch
        # get protein input
        target_x, target_edge_index, target_batch = data_pro.x, data_pro.edge_index, data_pro.batch

        # target_seq=data_pro.target

        # print('size')
        # print('mol_x', mol_x.size(), 'edge_index', mol_edge_index.size(), 'batch', mol_batch.size())
        # print('target_x', target_x.size(), 'target_edge_index', target_batch.size(), 'batch', target_batch.size())

        x = self.mol_conv1(mol_x, mol_edge_index)
        x = self.relu(x)

        # mol_edge_index, _ = dropout_adj(mol_edge_index, training=self.training)
        x = self.mol_conv2(x, mol_edge_index)
        x = self.relu(x)

        # mol_edge_index, _ = dropout_adj(mol_edge_index, training=self.training)
        x = self.mol_conv3(x, mol_edge_index)
        x = self.relu(x)
        x = gep(x, mol_batch)  # global pooling

        # flatten
        x = self.relu(self.mol_fc_g1(x))
        x = self.dropout(x)
        x = self.mol_fc_g2(x)
        x = self.dropout(x)

        xt = self.pro_conv1(target_x, target_edge_index)
        xt = self.relu(xt)

        # target_edge_index, _ = dropout_adj(target_edge_index, training=self.training)
        xt = self.pro_conv2(xt, target_edge_index)
        xt = self.relu(xt)

        # target_edge_index, _ = dropout_adj(target_edge_index, training=self.training)
        xt = self.pro_conv3(xt, target_edge_index)
        xt = self.relu(xt)

        # xt = self.pro_conv4(xt, target_edge_index)
        # xt = self.relu(xt)
        xt = gep(xt, target_batch)  # global pooling

        # flatten
        xt = self.relu(self.pro_fc_g1(xt))
        xt = self.dropout(xt)
        xt = self.pro_fc_g2(xt)
        xt = self.dropout(xt)

        print("sizes of out coming gnns",x.size(), xt.size())
        # concat
        xc = torch.cat((x, xt), 1)
        print('after concat',xc.size())
        # add some dense layers
        xc = self.fc1(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        xc = self.fc2(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        out = self.out(xc)
        return out




import os
import torch
import torch.optim as optim
from torch.nn import MSELoss
from sklearn.model_selection import KFold
import numpy as np
from tqdm import tqdm
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv, global_mean_pool as gep
from scipy.stats import pearsonr
import warnings
import itertools
from sklearn.model_selection import train_test_split


# Suppress FutureWarning related to torch.load
warnings.filterwarnings('ignore', category=FutureWarning)


# Define the load_sample function
def load_sample(path):
    # Load individual sample from file
    sample = torch.load(path)
    mol_data = sample[0]
    pro_data = sample[1]
    target = sample[2]

    # Convert dictionaries to Data objects if necessary
    if isinstance(mol_data, dict):
        mol_data = Data(**mol_data)
    if isinstance(pro_data, dict):
        pro_data = Data(**pro_data)

    # Ensure that 'x' attribute is set
    if not hasattr(mol_data, 'x') or mol_data.x is None:
        if hasattr(mol_data, 'features'):
            mol_data.x = mol_data.features
            del mol_data.features
        else:
            raise ValueError("mol_data does not have 'x' or 'features' attribute")

    if not hasattr(pro_data, 'x') or pro_data.x is None:
        if hasattr(pro_data, 'features'):
            pro_data.x = pro_data.features
            del pro_data.features
        else:
            raise ValueError("pro_data does not have 'x' or 'features' attribute")

    # Ensure 'x' is a float tensor
    if not isinstance(mol_data.x, torch.Tensor):
        mol_data.x = torch.tensor(mol_data.x)
    if not isinstance(pro_data.x, torch.Tensor):
        pro_data.x = torch.tensor(pro_data.x)

    if mol_data.x.dtype != torch.float:
        mol_data.x = mol_data.x.float()
    if pro_data.x.dtype != torch.float:
        pro_data.x = pro_data.x.float()

    # Adjust 'edge_index' for mol_data
    # Ensure 'edge_index' is a tensor of type torch.long
    if not isinstance(mol_data.edge_index, torch.Tensor):
        mol_data.edge_index = torch.tensor(mol_data.edge_index, dtype=torch.long)
    else:
        mol_data.edge_index = mol_data.edge_index.long()

    # Ensure 'edge_index' has shape [2, num_edges]
    if mol_data.edge_index.shape[0] != 2:
        mol_data.edge_index = mol_data.edge_index.t()

    # Adjust 'edge_index' for pro_data
    if not isinstance(pro_data.edge_index, torch.Tensor):
        pro_data.edge_index = torch.tensor(pro_data.edge_index, dtype=torch.long)
    else:
        pro_data.edge_index = pro_data.edge_index.long()

    if pro_data.edge_index.shape[0] != 2:
        pro_data.edge_index = pro_data.edge_index.t()

    # Set 'num_nodes' attribute to suppress warnings
    mol_data.num_nodes = mol_data.x.size(0)
    pro_data.num_nodes = pro_data.x.size(0)

    return (mol_data, pro_data, target)

# Define the batch_loader function
def batch_loader(file_list, sample_dir, batch_size):
    batch = []
    for idx, file_name in enumerate(file_list):
        sample_path = os.path.join(sample_dir, file_name)
        sample = load_sample(sample_path)
        batch.append(sample)
        if len(batch) == batch_size:
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch

# Define the evaluation metrics functions
def get_mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def get_ci(y_true, y_pred):
    """
    Compute the concordance index between true and predicted values.
    """
    pairs = itertools.combinations(range(len(y_true)), 2)
    c = 0
    s = 0
    for i, j in pairs:
        if y_true[i] != y_true[j]:
            s += 1
            if (y_true[i] < y_true[j] and y_pred[i] < y_pred[j]) or \
               (y_true[i] > y_true[j] and y_pred[i] > y_pred[j]):
                c += 1
            elif y_pred[i] == y_pred[j]:
                c += 0.5
    return c / s if s != 0 else 0

def get_pearson(y_true, y_pred):
    return pearsonr(y_true.flatten(), y_pred.flatten())[0]



# Suppress FutureWarning related to torch.load
warnings.filterwarnings('ignore', category=FutureWarning)


#calculate metrics on training and testing data each epoch ! 
def train_and_evaluate(sample_dir, num_epochs=100, test_size=0.2, lr=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Running on {device}.")

    sample_files = [f for f in os.listdir(sample_dir) if f.endswith('.pt')]

    # Split data into training and test sets
    train_files, test_files = train_test_split(sample_files, test_size=test_size, random_state=42)

    # Create a directory for the model checkpoint
    training_model_dir = os.path.join(sample_dir, 'TrainingModel')
    if not os.path.exists(training_model_dir):
        os.makedirs(training_model_dir)
        print(f"Created directory for checkpoints at {training_model_dir}")
    else:
        print(f"Using existing TrainingModel directory at {training_model_dir}")

    # Determine input feature dimensions from your data
    sample = load_sample(os.path.join(sample_dir, train_files[0]))
    mol_data = sample[0]
    pro_data = sample[1]

    num_features_mol = mol_data.x.size(1)
    num_features_pro = pro_data.x.size(1)

    # Initialize the GNN model with correct input dimensions
    model = GNNNet(
        num_features_mol=num_features_mol,
        num_features_pro=num_features_pro
    ).to(device)
    print(f"Model is on device: {next(model.parameters()).device}")

    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = MSELoss()

    # Initialize starting epoch
    start_epoch = 1

    # Check for existing checkpoints in TrainingModel directory
    existing_checkpoints = [f for f in os.listdir(training_model_dir)
                            if f.endswith('.pt') and f.startswith('model_epoch')]

    if existing_checkpoints:
        # Find the latest checkpoint based on epoch number
        latest_checkpoint = max(existing_checkpoints,
                                key=lambda x: int(x.split('_epoch')[1].split('.pt')[0]))
        checkpoint_path = os.path.join(training_model_dir, latest_checkpoint)
        print(f"Loading checkpoint from {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        loaded_epoch = checkpoint['epoch']
        start_epoch = loaded_epoch + 1
        print(f"Resuming training from epoch {start_epoch}")
    else:
        print("No checkpoint found, starting training from scratch.")

    # Lists to store metrics for plotting or analysis if needed
    train_mse_list = []
    train_ci_list = []
    train_pearson_list = []
    test_mse_list = []
    test_ci_list = []
    test_pearson_list = []

    # Training loop with progress bar over epochs
    for epoch in tqdm(range(start_epoch, num_epochs + 1),
                      desc="Training", unit="epoch"):
        model.train()
        running_loss = 0.0

        # Prepare batch loader
        batch_size = 200  # Adjust batch size as needed
        batch_loader_iter = batch_loader(train_files, sample_dir, batch_size=batch_size)

        for batch_samples in batch_loader_iter:
            mol_data_list = []
            pro_data_list = []
            target_list = []

            for sample in batch_samples:
                mol_data = sample[0]
                pro_data = sample[1]
                target = sample[2]

                mol_data_list.append(mol_data)
                pro_data_list.append(pro_data)
                target_list.append(target)

            mol_batch = Batch.from_data_list(mol_data_list).to(device)
            pro_batch = Batch.from_data_list(pro_data_list).to(device)
            target = torch.tensor(target_list, dtype=torch.float32).view(-1).to(device)

            optimizer.zero_grad()
            output = model(mol_batch, pro_batch)
            loss = loss_fn(output.view(-1), target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * len(batch_samples)

        avg_loss = running_loss / len(train_files)
        # Use tqdm.write() to print without interfering with the progress bar
        tqdm.write(f"Epoch {epoch}/{num_epochs}, Loss: {avg_loss:.4f}")

        # Save the model and optimizer states after each epoch
        checkpoint_filename = f"model_epoch{epoch}.pt"
        checkpoint_path = os.path.join(training_model_dir, checkpoint_filename)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, checkpoint_path)
        tqdm.write(f"Checkpoint saved at epoch {epoch}")

        # ---------------------------------------
        # Evaluate on training data
        # ---------------------------------------
        model.eval()
        total_train_preds, total_train_labels = [], []
        with torch.no_grad():
            batch_loader_iter = batch_loader(train_files, sample_dir, batch_size=batch_size)

            for batch_samples in batch_loader_iter:
                mol_data_list = []
                pro_data_list = []
                target_list = []

                for sample in batch_samples:
                    mol_data = sample[0]
                    pro_data = sample[1]
                    target = sample[2]

                    mol_data_list.append(mol_data)
                    pro_data_list.append(pro_data)
                    target_list.append(target)

                mol_batch = Batch.from_data_list(mol_data_list).to(device)
                pro_batch = Batch.from_data_list(pro_data_list).to(device)
                target = torch.tensor(target_list, dtype=torch.float32).view(-1).to(device)

                output = model(mol_batch, pro_batch)
                total_train_preds.append(output.cpu().numpy())
                total_train_labels.append(target.cpu().numpy())

        # Convert lists to numpy arrays for evaluation
        total_train_preds = np.concatenate(total_train_preds)
        total_train_labels = np.concatenate(total_train_labels)

        # Calculate metrics
        train_mse = get_mse(total_train_labels, total_train_preds)
        train_ci = get_ci(total_train_labels, total_train_preds)
        train_pearson = get_pearson(total_train_labels, total_train_preds)

        # Store metrics for analysis if needed
        train_mse_list.append(train_mse)
        train_ci_list.append(train_ci)
        train_pearson_list.append(train_pearson)

        # Print training metrics
        tqdm.write(f"Epoch {epoch}/{num_epochs} - Training Metrics: MSE: {train_mse:.4f}, CI: {train_ci:.4f}, Pearson: {train_pearson:.4f}")

        # ---------------------------------------
        # Evaluate on test data
        # ---------------------------------------
        total_test_preds, total_test_labels = [], []
        with torch.no_grad():
            batch_loader_iter = batch_loader(test_files, sample_dir, batch_size=batch_size)

            for batch_samples in batch_loader_iter:
                mol_data_list = []
                pro_data_list = []
                target_list = []

                for sample in batch_samples:
                    mol_data = sample[0]
                    pro_data = sample[1]
                    target = sample[2]

                    mol_data_list.append(mol_data)
                    pro_data_list.append(pro_data)
                    target_list.append(target)

                mol_batch = Batch.from_data_list(mol_data_list).to(device)
                pro_batch = Batch.from_data_list(pro_data_list).to(device)
                target = torch.tensor(target_list, dtype=torch.float32).view(-1).to(device)

                output = model(mol_batch, pro_batch)
                total_test_preds.append(output.cpu().numpy())
                total_test_labels.append(target.cpu().numpy())

        # Convert lists to numpy arrays for evaluation
        total_test_preds = np.concatenate(total_test_preds)
        total_test_labels = np.concatenate(total_test_labels)

        # Calculate metrics
        test_mse = get_mse(total_test_labels, total_test_preds)
        test_ci = get_ci(total_test_labels, total_test_preds)
        test_pearson = get_pearson(total_test_labels, total_test_preds)

        # Store metrics for analysis if needed
        test_mse_list.append(test_mse)
        test_ci_list.append(test_ci)
        test_pearson_list.append(test_pearson)

        # Print test metrics
        tqdm.write(f"Epoch {epoch}/{num_epochs} - Test Metrics: MSE: {test_mse:.4f}, CI: {test_ci:.4f}, Pearson: {test_pearson:.4f}")

        # Switch back to training mode for the next epoch
        model.train()

    # Final evaluation on test data (optional, since we have the metrics from the last epoch)
    print(f"\nFinal Evaluation Results - Test MSE: {test_mse:.4f}, CI: {test_ci:.4f}, Pearson: {test_pearson:.4f}")

    return train_mse_list, train_ci_list, train_pearson_list, test_mse_list, test_ci_list, test_pearson_list


if __name__ == "__main__":
    sample_dir = 'prepared_samples'  # Adjust the path to your samples directory
    num_epochs = 300  # Adjust the number of epochs as needed
    test_size = 0.2   # Proportion of the dataset to include in the test split
    learning_rate = 0.001  # Learning rate

    # Run the training function
    train_mse_list, train_ci_list, train_pearson_list, test_mse_list, test_ci_list, test_pearson_list = train_and_evaluate(
        sample_dir,
        num_epochs=num_epochs,
        test_size=test_size,
        lr=learning_rate
    )

    # Optionally, plot the metrics over epochs
    # For example, using matplotlib
    import matplotlib.pyplot as plt

    epochs = range(1, num_epochs + 1)

    plt.figure(figsize=(12, 4))

    # Plot MSE
    plt.subplot(1, 3, 1)
    plt.plot(epochs, train_mse_list, label='Train MSE')
    plt.plot(epochs, test_mse_list, label='Test MSE')
    plt.xlabel('Epoch')
    plt.ylabel('MSE')
    plt.title('MSE over Epochs')
    plt.legend()

    # Plot CI
    plt.subplot(1, 3, 2)
    plt.plot(epochs, train_ci_list, label='Train CI')
    plt.plot(epochs, test_ci_list, label='Test CI')
    plt.xlabel('Epoch')
    plt.ylabel('Concordance Index (CI)')
    plt.title('CI over Epochs')
    plt.legend()

    # Plot Pearson Correlation
    plt.subplot(1, 3, 3)
    plt.plot(epochs, train_pearson_list, label='Train Pearson')
    plt.plot(epochs, test_pearson_list, label='Test Pearson')
    plt.xlabel('Epoch')
    plt.ylabel('Pearson Correlation')
    plt.title('Pearson Correlation over Epochs')
    plt.legend()

    plt.tight_layout()
    plt.show()


import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch

def add_box(ax, x, y, width, height, text, color="lightblue"):
    """Draws a rounded box with a label."""
    box = FancyBboxPatch((x, y), width, height, boxstyle="round,pad=0.3", color=color, edgecolor="black")
    ax.add_patch(box)
    ax.text(x + width / 2, y + height / 2, text, ha="center", va="center", fontsize=10, weight="bold")

def add_arrow(ax, start, end):
    """Draws an arrow between two points."""
    arrow = FancyArrowPatch(start, end, arrowstyle="->", color="black", mutation_scale=15, lw=1.5)
    ax.add_patch(arrow)

def draw_gnn_architecture(output_path="GNNNet_Architecture.png"):
    """Draws the schematic of the GNNNet architecture."""
    fig, ax = plt.subplots(figsize=(14, 12))
    ax.set_xlim(0, 12)
    ax.set_ylim(-14, 12)
    ax.axis("off")

    # Molecular graph pipeline
    add_box(ax, 1, 10, 3, 1, "Molecular Features\n(78D)", "skyblue")
    add_box(ax, 1, 8, 3, 1, "GCNConv\n78→78", "lightgreen")
    add_arrow(ax, (2.5, 9.9), (2.5, 8.1))
    add_box(ax, 1, 6, 3, 1, "GCNConv\n78→156", "lightgreen")
    add_arrow(ax, (2.5, 7.9), (2.5, 6.1))
    add_box(ax, 1, 4, 3, 1, "GCNConv\n156→312", "lightgreen")
    add_arrow(ax, (2.5, 5.9), (2.5, 4.1))
    add_box(ax, 1, 2, 3, 1, "Global Pooling", "lightcoral")
    add_arrow(ax, (2.5, 3.9), (2.5, 2.1))
    add_box(ax, 1, 0, 3, 1, "FC\n312→1024", "gold")
    add_arrow(ax, (2.5, 1.9), (2.5, 0.1))
    add_box(ax, 1, -2, 3, 1, "FC\n1024→128", "gold")
    add_arrow(ax, (2.5, -0.1), (2.5, -1.9))

    # Protein graph pipeline
    add_box(ax, 8, 10, 3, 1, "Protein Features\n(54D)", "skyblue")
    add_box(ax, 8, 8, 3, 1, "GCNConv\n54→54", "lightgreen")
    add_arrow(ax, (9.5, 9.9), (9.5, 8.1))
    add_box(ax, 8, 6, 3, 1, "GCNConv\n54→108", "lightgreen")
    add_arrow(ax, (9.5, 7.9), (9.5, 6.1))
    add_box(ax, 8, 4, 3, 1, "GCNConv\n108→216", "lightgreen")
    add_arrow(ax, (9.5, 5.9), (9.5, 4.1))
    add_box(ax, 8, 2, 3, 1, "Global Pooling", "lightcoral")
    add_arrow(ax, (9.5, 3.9), (9.5, 2.1))
    add_box(ax, 8, 0, 3, 1, "FC\n216→1024", "gold")
    add_arrow(ax, (9.5, 1.9), (9.5, 0.1))
    add_box(ax, 8, -2, 3, 1, "FC\n1024→128", "gold")
    add_arrow(ax, (9.5, -0.1), (9.5, -1.9))

    # Concatenation and dense layers
    add_box(ax, 4.5, -5, 5, 1, "Concatenate\n(128+128)", "lightpink")
    add_arrow(ax, (2.5, -2.9), (4.5, -4.5))
    add_arrow(ax, (9.5, -2.9), (9.5, -4.5))
    add_box(ax, 4.5, -7, 5, 1, "FC\n256→1024", "gold")
    add_arrow(ax, (7, -6), (7, -6.9))
    add_box(ax, 4.5, -9, 5, 1, "FC\n1024→512", "gold")
    add_arrow(ax, (7, -8), (7, -8.9))
    add_box(ax, 4.5, -11, 5, 1, "Output Layer\n512→1", "lightgray")
    add_arrow(ax, (7, -10), (7, -10.9))

    # Title
    ax.text(6, 11.5, "Detailed Schematic of GNNNet Architecture", fontsize=16, weight="bold", ha="center")

    plt.tight_layout()
    plt.savefig(output_path)
    plt.show()

# Generate and save the diagram
draw_gnn_architecture("GNNNet_Architecture.png")


